# Remoção dos recursos
Este notebook detalha o processo de remover os recursos criados no notebook anterior "Filtro_de_promocao.ipynb" carregando as informações dos recuros já salvos.

(!) Se você criou recursos utilizando múltiplos notebooks e tem múltiplos grupos de conjuntos de dados, execute a célula que começa com `# store for cleanup` no final do notebook e logo em seguida execute este notebook para cada um deles (ex: caso tenha criado dois grupos de conjuntos de dados, terá que executar este notebook duas vezes). Este código remove apenas os recursos especificados em `# store for cleanup`, corresponderão ao último notebook executado).

Você pode utilizar este notebook para remover recursos do Amazon Personalize. Os ARNs dos recursos são definidos no notebook anterior, e também é possível adicioná-los de forma manual abaixo.

In [ ]:
%store -r

In [ ]:
# Se não puder/não quiser usar "%store -r" para carregar os recursos a serem apagados, 
# pode retirar o comentário do código abaixo e inseri-los manualmente

# dataset_group_arn='XXXXX'
# role_name='XXXXX'
# region='XXXXX'

Valide se as informações correspondem aos recursos a serem removidos. 
Importante: Esta operação não pode ser desfeita.

In [ ]:
print ('dataset_group_arn:', dataset_group_arn)
print ('role_name:', role_name)
print ('region:', region)

In [ ]:
schema_arns = []

In [ ]:
import sys
import getopt
import logging
import botocore
import boto3
import time
from packaging import version
from time import sleep
from botocore.exceptions import ClientError

logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

personalize = None

In [ ]:
def _delete_event_trackers(dataset_group_arn):
    event_tracker_arns = []

    event_trackers_paginator = personalize.get_paginator('list_event_trackers')
    for event_tracker_page in event_trackers_paginator.paginate(datasetGroupArn = dataset_group_arn):
        for event_tracker in event_tracker_page['eventTrackers']:
            if event_tracker['status'] in [ 'ACTIVE', 'CREATE FAILED' ]:
                logger.info('Removendo o event tracker {}'.format(event_tracker['eventTrackerArn']))
                personalize.delete_event_tracker(eventTrackerArn = event_tracker['eventTrackerArn'])
            elif event_tracker['status'].startswith('DELETE'):
                logger.warning('Event tracker {} esta sendo removido e vai aguardar até completar'.format(event_tracker['eventTrackerArn']))
            else:
                raise Exception('Solução {} esta com o status {} e não pode ser removida'.format(event_tracker['eventTrackerArn'], event_tracker['status']))

            event_tracker_arns.append(event_tracker['eventTrackerArn'])

    max_time = time.time() + 30*60 # 30 minutos
    while time.time() < max_time:
        for event_tracker_arn in event_tracker_arns:
            try:
                describe_response = personalize.describe_event_tracker(eventTrackerArn = event_tracker_arn)
                logger.debug('Event tracker {} esta com o status {}'.format(event_tracker_arn, describe_response['eventTracker']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    event_tracker_arns.remove(event_tracker_arn)

        if len(event_tracker_arns) == 0:
            logger.info('Todos os event trackers foram removidos ou não existem para o dataset group')
            break
        else:
            logger.info('Agurdando para {} event tracker(s) ser(em) removido(s)'.format(len(event_tracker_arns)))
            time.sleep(20)

    if len(event_tracker_arns) > 0:
        raise Exception('Timed out enquanto aguardava a remoção dos event trackers')

def _delete_filters(dataset_group_arn):
    filter_arns = []

    filters_response = personalize.list_filters(datasetGroupArn = dataset_group_arn, maxResults = 100)
    for filter in filters_response['Filters']:
        logger.info('Removendo filtro ' + filter['filterArn'])
        personalize.delete_filter(filterArn = filter['filterArn'])
        filter_arns.append(filter['filterArn'])

    max_time = time.time() + 30*60 # 30 minutos
    while time.time() < max_time:
        for filter_arn in filter_arns:
            try:
                describe_response = personalize.describe_filter(filterArn = filter_arn)
                logger.debug('Filtro {} esta com o status {}'.format(filter_arn, describe_response['filter']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    filter_arns.remove(filter_arn)

        if len(filter_arns) == 0:
            logger.info('Todos os filtros foram removidos ou não existem para o dataset group')
            break
        else:
            logger.info('Aguardando para {} filtro(s) ser(em) removido(s)'.format(len(filter_arns)))
            time.sleep(20)

    if len(filter_arns) > 0:
        raise Exception('Time out enquanto aguardava a remoção dos filtros')
        
def _delete_recommenders(dataset_group_arn):
    recommender_arns = []
    recommenders_response = personalize.list_recommenders(datasetGroupArn = dataset_group_arn, maxResults = 100)
    for recommender in recommenders_response['recommenders']:
        logger.info('Removendo recomendador ' + recommender['recommenderArn'])
        personalize.delete_recommender(recommenderArn = recommender['recommenderArn'])
        recommender_arns.append(recommender['recommenderArn'])
    max_time = time.time() + 30*60 # 30 minutos
    while time.time() < max_time:
        for recommender_arn in recommender_arns:
            try:
                describe_response = personalize.describe_recommender(recommenderArn = recommender_arn)
                logger.debug('Recomendados {} esta com o status {}'.format(recommender_arn, describe_response['recommender']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    recommender_arns.remove(recommender_arn)

        if len(recommender_arns) == 0:
            logger.info('Todos os recomendadores foram removidos ou não existem para o dataset group')
            break
        else:
            logger.info('Aguardando para {} recomendador(es) ser(em) removido(s)'.format(len(recommender_arns)))
            time.sleep(20)

    if len(recommender_arns) > 0:
        raise Exception('Timed out enquanto aguardava a remoção dos recomendadores')
    

def _delete_datasets_and_schemas(dataset_group_arn, schema_arns):
    dataset_arns = []
    
    dataset_paginator = personalize.get_paginator('list_datasets')
    for dataset_page in dataset_paginator.paginate(datasetGroupArn = dataset_group_arn):
        for dataset in dataset_page['datasets']:
            describe_response = personalize.describe_dataset(datasetArn = dataset['datasetArn'])
            schema_arns.append(describe_response['dataset']['schemaArn'])

            if dataset['status'] in ['ACTIVE', 'CREATE FAILED']:
                logger.info('Removendo conjunto de dados ' + dataset['datasetArn'])
                personalize.delete_dataset(datasetArn = dataset['datasetArn'])
            elif dataset['status'].startswith('DELETE'):
                logger.warning('Conjunto de dados {} esta sendo removido e vai aguardar até completar'.format(dataset['datasetArn']))
            else:
                raise Exception('Conjunto de dados {} esta com o status {} e não pode ser removido'.format(dataset['datasetArn'], dataset['status']))

            dataset_arns.append(dataset['datasetArn'])

    max_time = time.time() + 30*60 # 30 minutos
    while time.time() < max_time:
        for dataset_arn in dataset_arns:
            try:
                describe_response = personalize.describe_dataset(datasetArn = dataset_arn)
                logger.debug('Dataset {} status is {}'.format(dataset_arn, describe_response['dataset']['status']))
            except ClientError as e:
                error_code = e.response['Error']['Code']
                if error_code == 'ResourceNotFoundException':
                    dataset_arns.remove(dataset_arn)

        if len(dataset_arns) == 0:
            logger.info('Todos os conjuntos de dados foram removidos ou não existem para o dataset group')
            break
        else:
            logger.info('Aguardando para {} conjunto(s) de dados ser(em) removido(s)'.format(len(dataset_arns)))
            time.sleep(20)

    if len(dataset_arns) > 0:
        raise Exception('Timed out enquanto aguardava a remoção dos conjuntos de dados')

    for schema_arn in schema_arns:
        try:
            logger.info('Removendo o esquema ' + schema_arn)
            personalize.delete_schema(schemaArn = schema_arn)
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceInUseException':
                logger.info('Esquema {} ainda esta sendo utilizado por outro conjunto de dados'.format(schema_arn))
            else:
                raise e

    logger.info('Todos os esquemas utilizados por este conjunto de dados foram removidos ou não existem mais')

def _delete_dataset_group(dataset_group_arn):
    logger.info('Removendo o grupo de conjunto de dados ' + dataset_group_arn)
    personalize.delete_dataset_group(datasetGroupArn = dataset_group_arn)

    max_time = time.time() + 30*60 # 30 minutos
    while time.time() < max_time:
        try:
            describe_response = personalize.describe_dataset_group(datasetGroupArn = dataset_group_arn)
            logger.debug('Grupo de conjunto de dados {} status é {}'.format(dataset_group_arn, describe_response['datasetGroup']['status']))
            break
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceNotFoundException':
                logger.info('Grupo de conjunto de dados {} foi removido'.format(dataset_group_arn))
            else:
                raise e

        logger.info('Aguardando o grupo de conjunto de dados ser removido')
        time.sleep(20)

def delete_dataset_groups(dataset_group_arns, schema_arns, region = None):
    global personalize
    personalize = boto3.client(service_name = 'personalize', region_name = region)

    for dataset_group_arn in dataset_group_arns:
        logger.info('Dataset Group ARN: ' + dataset_group_arn)

        # 1. Remove recomendadores
        _delete_recommenders(dataset_group_arn)
        
        # 2. Remove event trackers
        _delete_event_trackers(dataset_group_arn)

        # 3. Remove filtros
        _delete_filters(dataset_group_arn)

        # 4. Remove conjuntos de dados e esquemas
        _delete_datasets_and_schemas(dataset_group_arn, schema_arns)

        # 5. Remove grupo de conjunto de dados
        _delete_dataset_group(dataset_group_arn)

        logger.info(f'Grupo de conjunto de dados {dataset_group_arn} foi removido')


In [ ]:
delete_dataset_groups([dataset_group_arn], schema_arns, region)

## Remoção da função IAM
Comece removendo a função IAM.

In [ ]:
iam = boto3.client('iam')

Identifique o nome da função que será removida.

Não é possível remover uma função de IAM que ainda tenha políticas anexadas. Então depois de identificar a função, vamos listar as políticas associadas a ela.

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

In [ ]:
# desanexar as políticas da função IAM
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

Em seguida, é possível remover a função IAM.

In [ ]:
iam.delete_role(
    RoleName = role_name
)

## Remoção do bucket S3
Para excluir um bucket do S3 ele precisa estar vazio. A maneira mais fácil de excluir um bucket S3 é navegar até o console do S3, excluir os objetos no bucket e depois excluir próprio bucket S3.